<a href="https://colab.research.google.com/github/arulrajgopal-zerotoone/zero_to_one_spark/blob/main/apache_spark/13_joins.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark

from pyspark.sql import SparkSession

#create spark session
spark= SparkSession.builder.appName('mysparksession').getOrCreate()

#create spark context
sc = spark.sparkContext

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.3-py2.py3-none-any.whl size=317840625 sha256=cd70f761d4092d19674d6455348464e1ae75628e1b8a95d4344b18f49f2304b1
  Stored in directory: /root/.cache/pip/wheels/1b/3a/92/28b93e2fbfdbb07509ca4d6f50c5e407f48dce4ddbda69a4ab
Successfully built pyspark


In [12]:
from pyspark.sql.functions import col,broadcast
from pyspark.sql.types import StructType, StructField, StringType, IntegerType


In [3]:
emp = [(1,"Smith",-1,"2018","10","M",3000), \
    (2,"Rose",1,"2010","20","M",4000), \
    (3,"Williams",1,"2010","10","M",1000), \
    (4,"Jones",2,"2005","10","F",2000), \
    (5,"Brown",2,"2010","40","",-1), \
      (6,"Brown",2,"2010","50","",-1) \
  ]
empColumns = ["emp_id","name","superior_emp_id","year_joined", \
       "emp_dept_id","gender","salary"]

empDF = spark.createDataFrame(data=emp, schema = empColumns)


dept = [("Finance",10), \
    ("Marketing",20), \
    ("Sales",30), \
    ("IT",40) \
  ]
deptColumns = ["dept_name","dept_id"]
deptDF = spark.createDataFrame(data=dept, schema = deptColumns)

empDF.printSchema()
deptDF.printSchema()
empDF.show(truncate=False)
deptDF.show(truncate=False)

root
 |-- emp_id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- superior_emp_id: long (nullable = true)
 |-- year_joined: string (nullable = true)
 |-- emp_dept_id: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary: long (nullable = true)

root
 |-- dept_name: string (nullable = true)
 |-- dept_id: long (nullable = true)

+------+--------+---------------+-----------+-----------+------+------+
|emp_id|name    |superior_emp_id|year_joined|emp_dept_id|gender|salary|
+------+--------+---------------+-----------+-----------+------+------+
|1     |Smith   |-1             |2018       |10         |M     |3000  |
|2     |Rose    |1              |2010       |20         |M     |4000  |
|3     |Williams|1              |2010       |10         |M     |1000  |
|4     |Jones   |2              |2005       |10         |F     |2000  |
|5     |Brown   |2              |2010       |40         |      |-1    |
|6     |Brown   |2              |2010       |50         |

## spark joins
1. inner
2. outer, full, fullouter, full_outer
3. left, leftouter, left_outer
4. right, rightouter, right_outer
5. anti, leftanti, left_anti
6. semi, leftsemi, left_semi

#inner

In [4]:
#inner join
empDF.join(deptDF,empDF.emp_dept_id ==  deptDF.dept_id,"inner") \
     .show(truncate=False)

+------+--------+---------------+-----------+-----------+------+------+---------+-------+
|emp_id|name    |superior_emp_id|year_joined|emp_dept_id|gender|salary|dept_name|dept_id|
+------+--------+---------------+-----------+-----------+------+------+---------+-------+
|1     |Smith   |-1             |2018       |10         |M     |3000  |Finance  |10     |
|3     |Williams|1              |2010       |10         |M     |1000  |Finance  |10     |
|4     |Jones   |2              |2005       |10         |F     |2000  |Finance  |10     |
|2     |Rose    |1              |2010       |20         |M     |4000  |Marketing|20     |
|5     |Brown   |2              |2010       |40         |      |-1    |IT       |40     |
+------+--------+---------------+-----------+-----------+------+------+---------+-------+



#outer

In [5]:
#outer
empDF.join(deptDF,empDF.emp_dept_id ==  deptDF.dept_id,"outer")\
    .show(truncate=False)


+------+--------+---------------+-----------+-----------+------+------+---------+-------+
|emp_id|name    |superior_emp_id|year_joined|emp_dept_id|gender|salary|dept_name|dept_id|
+------+--------+---------------+-----------+-----------+------+------+---------+-------+
|1     |Smith   |-1             |2018       |10         |M     |3000  |Finance  |10     |
|3     |Williams|1              |2010       |10         |M     |1000  |Finance  |10     |
|4     |Jones   |2              |2005       |10         |F     |2000  |Finance  |10     |
|2     |Rose    |1              |2010       |20         |M     |4000  |Marketing|20     |
|NULL  |NULL    |NULL           |NULL       |NULL       |NULL  |NULL  |Sales    |30     |
|5     |Brown   |2              |2010       |40         |      |-1    |IT       |40     |
|6     |Brown   |2              |2010       |50         |      |-1    |NULL     |NULL   |
+------+--------+---------------+-----------+-----------+------+------+---------+-------+



#right

In [6]:
# right
empDF.join(deptDF,empDF.emp_dept_id ==  deptDF.dept_id,"right") \
   .show(truncate=False)

+------+--------+---------------+-----------+-----------+------+------+---------+-------+
|emp_id|name    |superior_emp_id|year_joined|emp_dept_id|gender|salary|dept_name|dept_id|
+------+--------+---------------+-----------+-----------+------+------+---------+-------+
|4     |Jones   |2              |2005       |10         |F     |2000  |Finance  |10     |
|3     |Williams|1              |2010       |10         |M     |1000  |Finance  |10     |
|1     |Smith   |-1             |2018       |10         |M     |3000  |Finance  |10     |
|2     |Rose    |1              |2010       |20         |M     |4000  |Marketing|20     |
|NULL  |NULL    |NULL           |NULL       |NULL       |NULL  |NULL  |Sales    |30     |
|5     |Brown   |2              |2010       |40         |      |-1    |IT       |40     |
+------+--------+---------------+-----------+-----------+------+------+---------+-------+



#left

In [7]:
# left
empDF.join(deptDF,empDF.emp_dept_id ==  deptDF.dept_id,"left")\
    .show(truncate=False)

+------+--------+---------------+-----------+-----------+------+------+---------+-------+
|emp_id|name    |superior_emp_id|year_joined|emp_dept_id|gender|salary|dept_name|dept_id|
+------+--------+---------------+-----------+-----------+------+------+---------+-------+
|1     |Smith   |-1             |2018       |10         |M     |3000  |Finance  |10     |
|3     |Williams|1              |2010       |10         |M     |1000  |Finance  |10     |
|2     |Rose    |1              |2010       |20         |M     |4000  |Marketing|20     |
|6     |Brown   |2              |2010       |50         |      |-1    |NULL     |NULL   |
|4     |Jones   |2              |2005       |10         |F     |2000  |Finance  |10     |
|5     |Brown   |2              |2010       |40         |      |-1    |IT       |40     |
+------+--------+---------------+-----------+-----------+------+------+---------+-------+



#left anti & left semi

In [8]:
# left
empDF.join(deptDF,empDF.emp_dept_id ==  deptDF.dept_id,"left")\
      .filter(col("dept_id").isNull())\
      .drop("dept_name","dept_id")\
      .show(truncate=False)

+------+-----+---------------+-----------+-----------+------+------+
|emp_id|name |superior_emp_id|year_joined|emp_dept_id|gender|salary|
+------+-----+---------------+-----------+-----------+------+------+
|6     |Brown|2              |2010       |50         |      |-1    |
+------+-----+---------------+-----------+-----------+------+------+



In [9]:
# left anti join
empDF.join(deptDF,empDF.emp_dept_id ==  deptDF.dept_id,"leftanti")\
   .show(truncate=False)

+------+-----+---------------+-----------+-----------+------+------+
|emp_id|name |superior_emp_id|year_joined|emp_dept_id|gender|salary|
+------+-----+---------------+-----------+-----------+------+------+
|6     |Brown|2              |2010       |50         |      |-1    |
+------+-----+---------------+-----------+-----------+------+------+



In [10]:
# left_semi
empDF.join(deptDF,empDF.emp_dept_id ==  deptDF.dept_id,"leftsemi")\
   .show(truncate=False)

+------+--------+---------------+-----------+-----------+------+------+
|emp_id|name    |superior_emp_id|year_joined|emp_dept_id|gender|salary|
+------+--------+---------------+-----------+-----------+------+------+
|1     |Smith   |-1             |2018       |10         |M     |3000  |
|3     |Williams|1              |2010       |10         |M     |1000  |
|4     |Jones   |2              |2005       |10         |F     |2000  |
|2     |Rose    |1              |2010       |20         |M     |4000  |
|5     |Brown   |2              |2010       |40         |      |-1    |
+------+--------+---------------+-----------+-----------+------+------+



#join with multiple keys

In [13]:
record_1 = [1,'A','arul','cricket']
record_2 = [2,'A','sekar','chess']
record_3 = [3,'A','kumar','tennis']
record_4 = [1,'B', 'ganesh','football']
record_5 = [2,'B','vinoth','volleyball']
record_6 = [3,'B','Ravi','hockey']

record_6 = [1, 'A','Engineer']
record_7 = [2, 'A', 'doctor']
record_8 = [2,'B', 'lawyer']

list1 = [record_1, record_2, record_3,record_4,record_5]
list2 = [record_6, record_7, record_8]

df_schema = StructType(fields=[StructField("sr_no", IntegerType(), False),
                               StructField("section", StringType(), False),
                                StructField("name", StringType(), True),
                               StructField("fav_game", StringType(), True)
])

df_2_schema = StructType(fields=[StructField("sr_no", IntegerType(), False),
                                 StructField("section", StringType(), False),
                                StructField("profession", StringType(), True),
])

df = spark.createDataFrame(list1, df_schema)
df_2 = spark.createDataFrame(list2, df_2_schema)
df.show()
df_2.show()

+-----+-------+------+----------+
|sr_no|section|  name|  fav_game|
+-----+-------+------+----------+
|    1|      A|  arul|   cricket|
|    2|      A| sekar|     chess|
|    3|      A| kumar|    tennis|
|    1|      B|ganesh|  football|
|    2|      B|vinoth|volleyball|
+-----+-------+------+----------+

+-----+-------+----------+
|sr_no|section|profession|
+-----+-------+----------+
|    1|      A|  Engineer|
|    2|      A|    doctor|
|    2|      B|    lawyer|
+-----+-------+----------+



In [14]:
joined_df = df.alias('LH')\
                .join(df_2.alias('RH'), (col('LH.sr_no') == col('RH.sr_no')) & (col('LH.section') == col('RH.section')) , 'left')\
                .select('LH.*','RH.profession')

joined_df.show()
joined_df.explain(True)


+-----+-------+------+----------+----------+
|sr_no|section|  name|  fav_game|profession|
+-----+-------+------+----------+----------+
|    1|      A|  arul|   cricket|  Engineer|
|    2|      A| sekar|     chess|    doctor|
|    3|      A| kumar|    tennis|      NULL|
|    2|      B|vinoth|volleyball|    lawyer|
|    1|      B|ganesh|  football|      NULL|
+-----+-------+------+----------+----------+

== Parsed Logical Plan ==
'Project [LH.*, 'RH.profession]
+- Join LeftOuter, ((sr_no#416 = sr_no#424) AND (section#417 = section#425))
   :- SubqueryAlias LH
   :  +- LogicalRDD [sr_no#416, section#417, name#418, fav_game#419], false
   +- SubqueryAlias RH
      +- LogicalRDD [sr_no#424, section#425, profession#426], false

== Analyzed Logical Plan ==
sr_no: int, section: string, name: string, fav_game: string, profession: string
Project [sr_no#416, section#417, name#418, fav_game#419, profession#426]
+- Join LeftOuter, ((sr_no#416 = sr_no#424) AND (section#417 = section#425))
   :- Subq

#broadcast join
Note: - this is not large enough to broadcast, but for demo purpose, it has been done.

In [16]:
large_df = df
small_df = df_2

result_df = large_df.alias("LH").join(broadcast(small_df.alias("RH")), (col('LH.sr_no') == col('RH.sr_no')) & (col('LH.section') == col('RH.section')), "left")
result_df.show()


result_df.explain(True)


spark.conf.get("spark.sql.autoBroadcastJoinThreshold")


# To disable autoBroadcastJoin >> set -1
# By default it is 10485760 i.e. 10MB
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", -1)
spark.conf.get("spark.sql.autoBroadcastJoinThreshold")

+-----+-------+------+----------+-----+-------+----------+
|sr_no|section|  name|  fav_game|sr_no|section|profession|
+-----+-------+------+----------+-----+-------+----------+
|    1|      A|  arul|   cricket|    1|      A|  Engineer|
|    2|      A| sekar|     chess|    2|      A|    doctor|
|    3|      A| kumar|    tennis| NULL|   NULL|      NULL|
|    1|      B|ganesh|  football| NULL|   NULL|      NULL|
|    2|      B|vinoth|volleyball|    2|      B|    lawyer|
+-----+-------+------+----------+-----+-------+----------+

== Parsed Logical Plan ==
Join LeftOuter, ((sr_no#416 = sr_no#424) AND (section#417 = section#425))
:- SubqueryAlias LH
:  +- LogicalRDD [sr_no#416, section#417, name#418, fav_game#419], false
+- ResolvedHint (strategy=broadcast)
   +- SubqueryAlias RH
      +- LogicalRDD [sr_no#424, section#425, profession#426], false

== Analyzed Logical Plan ==
sr_no: int, section: string, name: string, fav_game: string, sr_no: int, section: string, profession: string
Join Left

'-1'